# Original Wide & Deep Tutorial

In [1]:
import tempfile
import pandas as pd
import urllib.request
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# Categorical base columns.
gender = tf.contrib.layers.sparse_column_with_keys(column_name="gender", keys=["Female", "Male"])
race = tf.contrib.layers.sparse_column_with_keys(column_name="race", keys=[
  "Amer-Indian-Eskimo", "Asian-Pac-Islander", "Black", "Other", "White"])
education = tf.contrib.layers.sparse_column_with_hash_bucket("education", hash_bucket_size=1000)
relationship = tf.contrib.layers.sparse_column_with_hash_bucket("relationship", hash_bucket_size=100)
workclass = tf.contrib.layers.sparse_column_with_hash_bucket("workclass", hash_bucket_size=100)
occupation = tf.contrib.layers.sparse_column_with_hash_bucket("occupation", hash_bucket_size=1000)
native_country = tf.contrib.layers.sparse_column_with_hash_bucket("native_country", hash_bucket_size=1000)

# Continuous base columns.
age = tf.contrib.layers.real_valued_column("age")
age_buckets = tf.contrib.layers.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
education_num = tf.contrib.layers.real_valued_column("education_num")
capital_gain = tf.contrib.layers.real_valued_column("capital_gain")
capital_loss = tf.contrib.layers.real_valued_column("capital_loss")
hours_per_week = tf.contrib.layers.real_valued_column("hours_per_week")

In [3]:
wide_columns = [
  gender, native_country, education, occupation, workclass, relationship, age_buckets,
  tf.contrib.layers.crossed_column([education, occupation], hash_bucket_size=int(1e4)),
  tf.contrib.layers.crossed_column([native_country, occupation], hash_bucket_size=int(1e4)),
  tf.contrib.layers.crossed_column([age_buckets, education, occupation], hash_bucket_size=int(1e6))]

In [4]:
deep_columns = [
  tf.contrib.layers.embedding_column(workclass, dimension=8),
  tf.contrib.layers.embedding_column(education, dimension=8),
  tf.contrib.layers.embedding_column(gender, dimension=8),
  tf.contrib.layers.embedding_column(relationship, dimension=8),
  tf.contrib.layers.embedding_column(native_country, dimension=8),
  tf.contrib.layers.embedding_column(occupation, dimension=8),
  age, education_num, capital_gain, capital_loss, hours_per_week]

In [6]:
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100, 50],
    fix_global_step_increment_bug=True)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_task_type': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10fb7ff60>, '_num_ps_replicas': 0, '_tf_random_seed': None, '_model_dir': '/var/folders/qv/glzl2pyj2g15vz3tn5s52c9w0000gn/T/tmpbqbm7dpx', '_environment': 'local', '_save_summary_steps': 100, '_evaluation_master': '', '_num_worker_replicas': 0, '_session_config': None, '_is_chief': True, '_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
}


In [7]:
# Define the column names for the data sets.
COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
  "marital_status", "occupation", "relationship", "race", "gender",
  "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_bracket"]
LABEL_COLUMN = 'label'
CATEGORICAL_COLUMNS = ["workclass", "education", "marital_status", "occupation",
                       "relationship", "race", "gender", "native_country"]
CONTINUOUS_COLUMNS = ["age", "education_num", "capital_gain", "capital_loss",
                      "hours_per_week"]

# Download the training and test data to temporary files.
# Alternatively, you can download them yourself and change train_file and
# test_file to your own paths.
train_file = tempfile.NamedTemporaryFile()
test_file = tempfile.NamedTemporaryFile()
urllib.request.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.data", train_file.name)
urllib.request.urlretrieve("http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/adult.test", test_file.name)

# Read the training and test data sets into Pandas dataframe.
df_train = pd.read_csv(train_file, names=COLUMNS, skipinitialspace=True)
df_test = pd.read_csv(test_file, names=COLUMNS, skipinitialspace=True, skiprows=1)
# If you want to do predictions locally. Put the data in a csv file in the same format as train/test file.
#df_predict = pd.read_csv(predict_file, names=COLUMNS, skipinitialspace=True)

df_train[LABEL_COLUMN] = (df_train['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)
df_test[LABEL_COLUMN] = (df_test['income_bracket'].apply(lambda x: '>50K' in x)).astype(int)

def input_fn(df, predict = False):
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values)
                       for k in CONTINUOUS_COLUMNS}
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(
        indices=[[i, 0] for i in range(df[k].size)],
        values=df[k].values,
        dense_shape=[df[k].size, 1])
                        for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols = dict(list(continuous_cols.items()) +
                        list(categorical_cols.items()))
    
    # Returns the feature columns and the label.
    if predict == True:
        return feature_cols
    else:
        # Converts the label column into a constant Tensor.
        label = tf.constant(df[LABEL_COLUMN].values)
        return feature_cols, label

def train_input_fn():
    return input_fn(df_train)

def eval_input_fn():
    return input_fn(df_test)

def predict_input_fn():
    return input_fn(df_predict, predict = True)

In [8]:
m.fit(input_fn=train_input_fn, steps=200)
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always 

INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-09-07-00:17:54
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.849579, accuracy/baseline_label_mean = 0.236226, accuracy/threshold_0.500000_mean = 0.849579, auc = 0.879309, auc_precision_recall = 0.749818, global_step = 200, labels/actual_label_mean = 0.236226, labels/prediction_mean = 0.219003, loss = 0.376738, precision/positive_threshold_0.500000_mean = 0.775325, recall/positive_threshold_0.500000_mean = 0.51144
accuracy: 0.849579
accuracy/baseline_label_mean: 0.236226
accuracy/threshold_0.500000_mean: 0.849579
auc: 0.879309
auc_precision_recall: 0.749818
global_step: 200
labels/actual_label_mean: 0.236226
labels/prediction_mean: 0.219003
loss: 0.376738
precision/positive_threshold_0.500000_mean: 0.775325
recall/positive_threshold_0.500000_mean: 0.51144


# Predict the Tensorflow Model Locally

In [9]:
# predict_file = "adult.predict.csv"
pre_results = m.predict_classes(input_fn=predict_input_fn)
pre_results_prob = m.predict_proba(input_fn=predict_input_fn)
for key,pro in zip(pre_results,pre_results_prob):
    print "%s : %s"%(key,[round(p,5) for p in pro])

0 : [0.80336, 0.19664]
0 : [0.61142, 0.38858]
0 : [0.79919, 0.20081]
1 : [0.00084, 0.99916]
0 : [0.85656, 0.14344]
0 : [0.74078, 0.25922]
0 : [0.75243, 0.24757]
0 : [0.55772, 0.44228]


# Export TensorFlow Model using .export_savedmodel()

In [10]:
# Import this function from wherever it will end up in the future
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils
from tensorflow.contrib.layers import create_feature_spec_for_parsing

In [11]:
# I step
feature_columns = wide_columns + deep_columns

# II step
feature_spec = create_feature_spec_for_parsing(feature_columns)

# III step
export_input_fn = input_fn_utils.build_parsing_serving_input_fn(feature_spec)

# IV step
servable_model_dir = "./serving_savemodel"
servable_model_path = m.export_savedmodel(servable_model_dir, export_input_fn)
servable_model_path

Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always 

b'./serving_savemodel/1504743714'